In [30]:

import xlwings as xw
import os
import pandas as pd
import numpy as np
pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [31]:
# --  Choose which file to import from below
extended_df_name = 'extended_fmeca.xlsx' # The whole fmeca - which has all the newly calculated columns
req_calc_name = 'fmeca_req_calc.xlsx' # A slice of the fmeca with selected columns that require calculation. The order of columns is important here.
correct_abts_name = 'correct_abts.xlsx' # A slice of the fmeca which requires abt corrections.
correct_dim_name = 'correct_dim.xlsx' # A slice of fmeca which requires correct diminution levels

# -- File path to import from
file_location = 'C:/Users/nisha/OneDrive - Floating Solutions Consulting/Documents/04. Projects/JAD-01 MV Main Deck/03. Working/FMECA/Batch 5 FMECA 2022/FMECA spreadsheets/AN_working/ProcessedData'

In [32]:
df = pd.read_excel(os.path.join(file_location,req_calc_name),sheet_name='Sheet1')

In [33]:
df.head(1)

,Stiffener(s),stiff_anom_combo,stiffener_dim,h,grade,ps,Average UTM Reading (mm),tp,tw,tf,"Nearest transverse member (Fr., TWF,TBHD)",Longitudinal\n(l) (distance\nfrom in mm),Frame(s)
0,BP,BP_4SWBT-LBHD- CR-545,350x19.5 + 0x0 L2,0.0,AH36,Sec,14.4,14.4,NaN,NaN,TB,0,FR-47


In [34]:
def renameColumns():
    df.rename(columns={"Nearest transverse member (Fr., TWF,TBHD)":"Nearest_transverse","Longitudinal\n(l) (distance\nfrom in mm)":"Distance_to_transverse","Frame(s)":"Frame"},inplace=True)
    return None

renameColumns()

In [36]:
df["Frame"].unique()

array(['FR-47', 'FR-46', 'FR-49', 'FR-48', 'TWF47-48', 'TWF54-55',
       'TWF48-49', 'TWF50-51', 'TWF54', 'TWF46-47', '49-50', '52-53'],
      dtype=object)

In [37]:
existing_map = ['FR-47', 'FR-46', 'FR-49', 'FR-48', 'TWF47-48', 'TWF54-55',
       'TWF48-49', 'TWF50-51', 'TWF54', 'TWF46-47', '49-50', '52-53']
new_map = ['FR-47', 'FR-46', 'FR-49', 'FR-48', 'FR-48', 'FR-55',
       'FR-49', 'FR-51', 'FR-54', 'FR-47', 'FR-50', 'FR-53']

map_dict = dict(zip(existing_map,new_map))

In [38]:
df["Frame"] = df["Frame"].map(map_dict)

In [39]:
df[df["Nearest_transverse"]=='TB']

,Stiffener(s),stiff_anom_combo,stiffener_dim,h,grade,ps,Average UTM Reading (mm),tp,tw,tf,Nearest_transverse,Distance_to_transverse,Frame
0,BP,BP_4SWBT-LBHD- CR-545,350x19.5 + 0x0 L2,0.00,AH36,Sec,14.400000,14.400000,NaN,NaN,TB,0,FR-47
186,LL23,LL23_4SWBT-LBHD- CR-450,500x10.5 + 150x23 L2,4.49,AH36,Sec,11.566667,11.566667,NaN,NaN,TB,100,FR-47
187,LL23,LL23_4SWBT-LBHD- CR-451,500x10.5 + 150x23 L2,4.49,AH36,Sec,12.400000,12.400000,NaN,NaN,TB,530,FR-47
188,LL23,LL23_4SWBT-LBHD- CR-452,500x10.5 + 150x23 L2,4.49,AH36,Sec,0.000000,0.000000,NaN,NaN,TB,210,FR-47
189,LL23,LL23_4SWBT-LBHD- CR-453,500x10.5 + 150x23 L2,4.49,AH36,Sec,11.400000,11.400000,NaN,NaN,TB,0,FR-47
231,LL27,LL27_4SWBT-LBHD- CR-538,350x10 + 100x14 L2,1.00,AH36,Sec,6.500000,NaN,6.5,NaN,TB,530,FR-47


### Changing frames to the nearest defined frame in global loads calculation

Not all the frames are defined in the global loads calculation. 


Therefore, either it has to be intrapolated or the existing frame has to be changed to the nearest defined frame such that the bending moment of the frame can be applied in the calculations.

In the below script a problem specific solution has been implemented. A more general code can be written to replace it with the nearest defined Frame name.

In [41]:
# -- Existing cleaned frame names
a = ['FR-47', 'FR-46', 'FR-49', 'FR-48', 'FR-48', 'FR-55',
       'FR-49', 'FR-51', 'FR-54', 'FR-47', 'FR-50', 'FR-53'] 

# -- Nearest defined frames
b = ['46', '46', '51', '46', '46', '56',
       '51', '51', '56', '46', '51', '51']

map_nearest_frame = dict(zip(a,b))

In [42]:
df['defined_frame'] = df['Frame'].map(map_nearest_frame)

In [43]:
df.head(4)

,Stiffener(s),stiff_anom_combo,stiffener_dim,h,grade,ps,Average UTM Reading (mm),tp,tw,tf,Nearest_transverse,Distance_to_transverse,Frame,defined_frame
0,BP,BP_4SWBT-LBHD- CR-545,350x19.5 + 0x0 L2,0.000,AH36,Sec,14.40,14.4,NaN,NaN,TB,0,FR-47,46
1,LL00,LL00_4SWBT-LBHD- CR-219,350x10 + 150x15.5 L2,23.497,AH36,Sec,6.99,NaN,6.99,NaN,FR-47,0,FR-46,46
2,LL00,LL00_4SWBT-LBHD- CR-220,350x10 + 150x15.5 L2,23.497,AH36,Sec,6.60,NaN,6.60,NaN,FR-47,650,FR-46,46
3,LL00,LL00_4SWBT-LBHD- CR-221,350x10 + 150x15.5 L2,23.497,AH36,Sec,6.30,NaN,6.30,NaN,FR-47,650,FR-46,46


In [44]:
xw.view(df)